In [1]:
import os; os.chdir("../")

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [1]:
record = []
dfs = []
for f in os.walk("results"):
    if len(f[-1]) > 0:
        info = f[0]
        if len(info.split("/")) < 5:
            continue
        _, dataset, model, framework, explainer = info.split("/")

        abpc = np.load(info + "/abpc.npy").mean()
        cmpx = np.load(info + "/cmpx.npy").mean()
        cmpd = np.load(info + "/cmpd.npy").mean()
        
        record.append([dataset, model, framework, explainer, abpc, cmpx, cmpd])
        print(f"Dataset: {dataset}, Model: {model}, Framework: {framework}, Explainer: {explainer}, ABPC: {abpc}, CMPX: {cmpx}, CMPD: {cmpd}")

record = pd.DataFrame(record, columns=["dataset", "model", "framework", "explainer", "abpc", "cmpx", "cmpd"])

NameError: name 'os' is not defined

In [4]:
record = []
dfs = []
for f in os.walk("results"):
    if len(f[-1]) > 0:
        info = f[0]
        res = info.split("/")
        if "pnpxai" in info:
            if len(res) == 4:
                _, dataset, model, framework = info.split("/")
            else:
                continue
        else:
            _, dataset, model, framework, explainer = info.split("/")

        if framework == "pnpxai":
            df = pd.read_csv(info + "/evaluation.csv")
            df['dataset'] = dataset
            df['model'] = model
            df['framework'] = framework
            dfs.append(df)
        else:
            abpc = np.load(info + "/abpc.npy").mean()
            cmpx = np.load(info + "/cmpx.npy").mean()
            cmpd = np.load(info + "/cmpd.npy").mean()
            
            record.append([dataset, model, framework, explainer, abpc, cmpx, cmpd])
            print(f"Dataset: {dataset}, Model: {model}, Framework: {framework}, Explainer: {explainer}, ABPC: {abpc}, CMPX: {cmpx}, CMPD: {cmpd}")

df = pd.concat(dfs, ignore_index=True).set_index(['dataset', 'model', 'framework', 'explainer', 'metric'])['value'].unstack().reset_index()
record = pd.DataFrame(record, columns=["dataset", "model", "framework", "explainer", "abpc", "cmpx", "cmpd"])
record = pd.concat([record, df], axis=0).reset_index(drop=True)

Dataset: Wine Quality, Model: xgb, Framework: captum, Explainer: shap, ABPC: 0.05462024357353095, CMPX: 1.7677967915168176, CMPD: -0.49210489177933103
Dataset: Wine Quality, Model: xgb, Framework: captum, Explainer: lime, ABPC: 0.05566922517454381, CMPX: 0.7818828447506978, CMPD: -0.19559640796138691
Dataset: Wine Quality, Model: xgb, Framework: omnixai, Explainer: shap, ABPC: 0.07486488741967011, CMPX: 1.7537420422809484, CMPD: -0.473717192950157
Dataset: Wine Quality, Model: xgb, Framework: omnixai, Explainer: lime, ABPC: 0.046090794249502534, CMPX: 1.8210483955772743, CMPD: -0.5140509629937319
Dataset: Wine Quality, Model: xgb, Framework: autoxai, Explainer: shap, ABPC: -0.0023007076151043294, CMPX: 1.735064955999162, CMPD: -0.5221299803256989
Dataset: Wine Quality, Model: xgb, Framework: autoxai, Explainer: lime, ABPC: -0.013010208403211437, CMPX: 1.8187786011963576, CMPD: -0.5547407251596451
Dataset: Wine Quality, Model: tab_resnet, Framework: captum, Explainer: shap, ABPC: 0.1167

In [5]:
EXP_MAP = {
    "shap" : "KernelSHAP",
    "kernel_shap" : "KernelSHAP",
    "lime" : "LIME",
    "integrated_gradients" : "Integrated Gradients",
    "itg": "Gradient x Input",
    "grad": "Gradient",
    "gradient": "Gradient",
    "lrp_uniform_epsilon" : "LRP",
    "lrp" : "LRP",
    "smooth_grad" : "SmoothGrad",
    "sg" : "SmoothGrad",
    "var_grad" : "VarGrad",
    "grad_x_input" : "Gradient x Input",
    "ig" : "Integrated Gradients",
}

MODEL_MAP = {
    "xgb" : "XGBoost",
    "tab_resnet": "Resnet",
}

FRAMEWORK_MAP = {
    "pnpxai" : "PnPXAI",
    "captum" : "Captum",
    "autoxai" : "AutoXAI",
    "omnixai" : "OmniXAI",
    "openxai": "OpenXAI",
}

METRIC_MAP = {
    "abpc": "Correctness", "cmpx": "Complexity", "cmpd": "Compounded"
}

NAME_MAP = EXP_MAP.copy()
NAME_MAP.update(MODEL_MAP)
NAME_MAP.update(FRAMEWORK_MAP)
NAME_MAP.update(METRIC_MAP)

In [6]:
from pandas.api.types import CategoricalDtype

exp_order = [
 'KernelSHAP',
 'LIME',
 'Gradient',
 'Gradient x Input',
 'Integrated Gradients',
 'SmoothGrad',
 'VarGrad',
 'LRP',
]


performance = (
    record
    .drop(columns='dataset')
    .replace(NAME_MAP)
    .melt(id_vars=["model", "framework", "explainer"],   # 그대로 둘 열
            value_vars=["abpc", "cmpx", "cmpd"],           # 녹일 열
            var_name="metric",                             # 새 열 이름
            value_name="value")
    .replace(NAME_MAP)
    .assign(
        model=lambda d: d["model"].astype(CategoricalDtype(list(MODEL_MAP.values()), ordered=True)),
        metric=lambda d: d["metric"].astype(CategoricalDtype(list(METRIC_MAP.values()), ordered=True)),
        explainer=lambda d: d["explainer"].astype(CategoricalDtype(exp_order, ordered=True)),
        framework=lambda d: d["framework"].astype(CategoricalDtype(list(FRAMEWORK_MAP.values()), ordered=True))
    )
    [['model', 'metric', 'explainer', 'framework', 'value']]
    .pivot(
        index=["model", "metric", "explainer"],  # 행 기준
        columns="framework",                    # 컬럼으로 펼칠 대상
        values="value"                          # 값
    )
    .reset_index()
).sort_values(["model", "metric", "explainer"]) #.drop(columns=['framework'])

In [7]:
print(performance.to_markdown())

|    | model   | metric      | explainer            |     PnPXAI |       Captum |      AutoXAI |     OmniXAI |      OpenXAI |
|---:|:--------|:------------|:---------------------|-----------:|-------------:|-------------:|------------:|-------------:|
|  0 | XGBoost | Correctness | KernelSHAP           |  0.14236   |   0.0546202  |  -0.00230071 |   0.0748649 | nan          |
|  1 | XGBoost | Correctness | LIME                 |  0.129031  |   0.0556692  |  -0.0130102  |   0.0460908 | nan          |
|  2 | XGBoost | Complexity  | KernelSHAP           |  1.62756   |   1.7678     |   1.73506    |   1.75374   | nan          |
|  3 | XGBoost | Complexity  | LIME                 |  0.604617  |   0.781883   |   1.81878    |   1.82105   | nan          |
|  4 | XGBoost | Compounded  | KernelSHAP           | -0.422027  |  -0.492105   |  -0.52213    |  -0.473717  | nan          |
|  5 | XGBoost | Compounded  | LIME                 | -0.12131   |  -0.195596   |  -0.554741   |  -0.514051  | nan    

In [8]:


(
    performance
            .replace({"metric" : {"abpc": "Correctness", "cmpx": "Complexity", "cmpd": "Compounded"}})
            .assign(metric = performance["metric"].astype(CategoricalDtype(categories=["Correctness", "Complexity", "Compounded"], ordered=True)))
            # .assign(metric = performance["metric"].astype(CategoricalDtype(categories=["Correctness", "Complexity", "Compounded"], ordered=True)))
            # .set_index(["model", "metric", "explainer", "framework"])  # 4단계 MultiIndex
            # ["value"]                # Series 로 만들어야 unstack 적용이 간단
            # .unstack("framework")    # 'framework' 레벨을 컬럼으로 펼치기
            # .reset_index()
            .round(4)
)


framework,model,metric,explainer,PnPXAI,Captum,AutoXAI,OmniXAI,OpenXAI
0,XGBoost,Correctness,KernelSHAP,0.1424,0.0546,-0.0023,0.0749,NaN
1,XGBoost,Correctness,LIME,0.1290,0.0557,-0.0130,0.0461,NaN
2,XGBoost,Complexity,KernelSHAP,1.6276,1.7678,1.7351,1.7537,NaN
3,XGBoost,Complexity,LIME,0.6046,0.7819,1.8188,1.8210,NaN
4,XGBoost,Compounded,KernelSHAP,-0.4220,-0.4921,-0.5221,-0.4737,NaN
5,XGBoost,Compounded,LIME,-0.1213,-0.1956,-0.5547,-0.5141,NaN
6,Resnet,Correctness,KernelSHAP,0.1289,0.1168,-0.0070,NaN,0.1360
7,Resnet,Correctness,LIME,0.1147,0.1078,0.0025,NaN,-0.0091
8,Resnet,Correctness,Gradient,0.0395,-0.0095,NaN,NaN,-0.0095
9,Resnet,Correctness,Gradient x Input,0.1306,0.1306,NaN,NaN,0.1306


In [9]:
print(performance.round(4).to_markdown())

|    | model   | metric      | explainer            |   PnPXAI |   Captum |   AutoXAI |   OmniXAI |   OpenXAI |
|---:|:--------|:------------|:---------------------|---------:|---------:|----------:|----------:|----------:|
|  0 | XGBoost | Correctness | KernelSHAP           |   0.1424 |   0.0546 |   -0.0023 |    0.0749 |  nan      |
|  1 | XGBoost | Correctness | LIME                 |   0.129  |   0.0557 |   -0.013  |    0.0461 |  nan      |
|  2 | XGBoost | Complexity  | KernelSHAP           |   1.6276 |   1.7678 |    1.7351 |    1.7537 |  nan      |
|  3 | XGBoost | Complexity  | LIME                 |   0.6046 |   0.7819 |    1.8188 |    1.821  |  nan      |
|  4 | XGBoost | Compounded  | KernelSHAP           |  -0.422  |  -0.4921 |   -0.5221 |   -0.4737 |  nan      |
|  5 | XGBoost | Compounded  | LIME                 |  -0.1213 |  -0.1956 |   -0.5547 |   -0.5141 |  nan      |
|  6 | Resnet  | Correctness | KernelSHAP           |   0.1289 |   0.1168 |   -0.007  |  nan      |    0

In [10]:
STOP

NameError: name 'STOP' is not defined

# Debugging

In [ ]:
import torch
dataset_nm = "Wine Quality"
model = "tab_resnet"

res = []

for framework in ["pnpxai", "captum"]:
    for explainer in ["grad", "itg"]:
        path = f"results/{dataset_nm}/{model}/{framework}/{explainer}"
        explanation = torch.tensor(np.load(f"{path}/explanations.npy"))
        res.append({
            "framework": framework,
            "explainer": explainer,
            "explanation": explanation,
        })

In [ ]:
idx = 0
print(f"Explainer: {res[idx]['explainer']}, Framework: {res[idx]['framework']}")
res[idx]['explanation']

Explainer: grad, Framework: pnpxai


tensor([[-0.0196,  0.0445,  0.0006,  ..., -0.0225, -0.0353, -0.0791],
        [-0.0502,  0.0766,  0.0242,  ...,  0.0281, -0.0069, -0.0653],
        [-0.0078,  0.0260,  0.0166,  ..., -0.0108, -0.0199, -0.0532],
        ...,
        [-0.0438,  0.0348, -0.0301,  ...,  0.0114, -0.0845, -0.1652],
        [-0.0686, -0.0778, -0.1023,  ..., -0.0596, -0.0766, -0.1172],
        [ 0.0016,  0.0557,  0.0250,  ..., -0.0628, -0.0487, -0.0953]])

In [ ]:
idx = 2
print(f"Explainer: {res[idx]['explainer']}, Framework: {res[idx]['framework']}")
res[idx]['explanation'], res[idx]['explanation'].shape

Explainer: grad, Framework: captum


(tensor([[0.0196, 0.0445, 0.0006,  ..., 0.0225, 0.0353, 0.0791],
         [0.0502, 0.0766, 0.0242,  ..., 0.0281, 0.0069, 0.0653],
         [0.0078, 0.0260, 0.0166,  ..., 0.0108, 0.0199, 0.0532],
         ...,
         [0.0438, 0.0348, 0.0301,  ..., 0.0114, 0.0845, 0.1652],
         [0.0686, 0.0778, 0.1023,  ..., 0.0596, 0.0766, 0.1172],
         [0.0016, 0.0557, 0.0250,  ..., 0.0628, 0.0487, 0.0953]]),
 torch.Size([1300, 11]))

In [ ]:
import numpy as np
dataset_nm = "Wine Quality"
X_test = np.load(f"data/{dataset_nm}/X_test.npy")

In [ ]:
X_test[0]

array([-0.86035472,  0.42724077, -0.54115347, -0.80783726, -0.20079018,
       -0.81841851,  0.37608571, -0.56583843,  0.63131247, -0.21014415,
       -0.07697409])

In [ ]:
res[0]['explanation'][0]

tensor([-0.0196,  0.0445,  0.0006, -0.0678,  0.0001, -0.0545,  0.0386,  0.0396,
        -0.0225, -0.0353, -0.0791])

In [ ]:
torch.tensor(X_test[0], dtype=torch.float32) * res[0]['explanation'][0]

tensor([ 1.6856e-02,  1.9002e-02, -3.3168e-04,  5.4808e-02, -2.9940e-05,
         4.4633e-02,  1.4526e-02, -2.2387e-02, -1.4206e-02,  7.4102e-03,
         6.0889e-03])

In [ ]:
idx = 1
print(f"Explainer: {res[idx]['explainer']}, Framework: {res[idx]['framework']}")
res[idx]['explanation']

Explainer: itg, Framework: pnpxai


tensor([[ 0.0169,  0.0190, -0.0003,  ..., -0.0142,  0.0074,  0.0061],
        [-0.0188, -0.0185,  0.0169,  ..., -0.0067, -0.0046,  0.0707],
        [-0.0053, -0.0094,  0.0196,  ...,  0.0060,  0.0162,  0.0219],
        ...,
        [ 0.0309, -0.0379,  0.0204,  ...,  0.0278, -0.0106,  0.0266],
        [ 0.0431,  0.1014,  0.0201,  ..., -0.0043,  0.1088, -0.1482],
        [ 0.0011, -0.0134,  0.0294,  ...,  0.0463,  0.0757, -0.0566]])

In [ ]:
idx = 3
print(f"Explainer: {res[idx]['explainer']}, Framework: {res[idx]['framework']}")
res[idx]['explanation'][0]

Explainer: itg, Framework: captum


tensor([ 0.0518,  0.0390,  0.0050,  0.0917, -0.0018,  0.0692,  0.0223, -0.0456,
        -0.0242,  0.0165,  0.0138], dtype=torch.float64)

In [ ]:
res = {}
fm, exp = "openxai", "itg"
path = f"results/Wine Quality/tab_resnet/{fm}/{exp}/explanations.npy"
res[(fm, exp)] = np.load(path)
fm, exp = "pnpxai", "ig"
path = f"results/Wine Quality/tab_resnet/{fm}/{exp}/explanations.npy"
res[(fm, exp)] = np.load(path)
fm, exp = "pnpxai", "itg"
path = f"results/Wine Quality/tab_resnet/{fm}/{exp}/explanations.npy"
res[(fm, exp)] = np.load(path)

In [ ]:
res

{('openxai',
  'itg'): array([[ 0.01685624,  0.01900235, -0.00033168, ..., -0.01420627,
          0.00741018,  0.00608889],
        [-0.01876599, -0.01845306,  0.01691097, ..., -0.00672095,
         -0.00455256,  0.07071263],
        [-0.00533765, -0.00940971,  0.01960211, ...,  0.00595997,
          0.01619228,  0.02192073],
        ...,
        [ 0.05618482,  0.03911267,  0.09194048, ..., -0.03664063,
         -0.02614157, -0.12047222],
        [ 0.02399565, -0.05562751,  0.00928165, ...,  0.12375217,
         -0.02626246, -0.14539205],
        [-0.00028029,  0.00899973,  0.00513067, ..., -0.00110954,
         -0.00156354,  0.00606695]], dtype=float32),
 ('pnpxai',
  'ig'): array([[ 0.01685624,  0.01900235, -0.00033168, ..., -0.01420627,
          0.00741018,  0.00608889],
        [-0.01876599, -0.01845306,  0.01691097, ..., -0.00672095,
         -0.00455256,  0.07071263],
        [-0.00533765, -0.00940971,  0.01960211, ...,  0.00595997,
          0.01619228,  0.02192073],
        ..

In [ ]:
fms = ['openxai', 'pnpxai']
exps = ['grad', 'ig']

res = {}
for fm in fms:
    for exp in exps:
        path = f"results/Wine Quality/tab_resnet/{fm}/{exp}/explanations.npy"
        res[(fm, exp)] = np.load(path)


In [ ]:
res[("openxai", "ig")].shape, res[("pnpxai", "ig")].shape

((100, 11), (100, 11))

In [ ]:
res[("openxai", "grad")][0], res[("pnpxai", "grad")][0]

(array([-0.0195922 ,  0.04447692,  0.00061291, -0.06784541,  0.00014911,
        -0.05453618,  0.0386231 ,  0.03956512, -0.02250276, -0.03526237,
        -0.07910307], dtype=float32),
 array([-0.0195922 ,  0.04447692,  0.00061291, -0.06784541,  0.00014911,
        -0.05453618,  0.0386231 ,  0.03956512, -0.02250276, -0.03526237,
        -0.07910307], dtype=float32))

In [ ]:
res[("openxai", "itg")][0], res[("pnpxai", "itg")][0]

(array([ 1.6856240e-02,  1.9002352e-02, -3.3167712e-04,  5.4808050e-02,
        -2.9940065e-05,  4.4633418e-02,  1.4525595e-02, -2.2387469e-02,
        -1.4206271e-02,  7.4101803e-03,  6.0888864e-03], dtype=float32),
 array([ 0.03004287,  0.04556599,  0.00657321,  0.05131613, -0.00039258,
         0.03146624,  0.01635183, -0.01861997, -0.0259457 , -0.00207138,
        -0.02360975]))

In [ ]:
fm = "pnpxai"
exp = "ig"
path = f"results/Wine Quality/tab_resnet/{fm}/{exp}/explanations.npy"
a = np.load(path)

In [ ]:
a.shape

(11,)

In [ ]:
fms = ['openxai', 'pnpxai']
exps = ['grad', 'shap']

res = {}
for fm in fms:
    for exp in exps:
        path = f"results/Wine Quality/tab_resnet/{fm}/{exp}/abpc.npy"
        res[(fm, exp)] = np.load(path)

In [ ]:
res[("openxai", "shap")][0], res[("pnpxai", "shap")][0]

(0.07961373776197433, 0.0607578344643116)

In [ ]:
res[("openxai", "shap")].mean(), res[("pnpxai", "shap")].mean()

(0.12462404012680053, 0.1056765974353766)

In [ ]:
res[("openxai", "grad")][0], res[("pnpxai", "grad")][0]

(-0.0360502190887928, -0.0360502190887928)

In [ ]:
res[("openxai", "grad")][0], res[("pnpxai", "grad")]

IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
res[("openxai", "grad")].mean(), res[("pnpxai", "grad")].mean()

(-0.009568325662403367, -0.0360502190887928)